In [69]:
!git add NER.ipynb
!git commit -m 'save csv'
!git push

[main c4abbae] save csv
 1 file changed, 541 insertions(+), 45 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 6.17 KiB | 6.17 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/gaylejuntilla/MA3831-A3.git
   642de08..c4abbae  main -> main


In [ ]:
import spacy
import numpy as np
import pandas as pd

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
corpus = pd.read_csv('corpus.csv')
corpus_df = corpus.copy()

In [48]:
party_leader_pairs = {
    "Australian Labor Party": "Anthony Albanese",
    "Liberal Party": "Peter Dutton",
    "National Party": "David Littleproud",
    "Coalition": "Peter Dutton",
    "Australian Greens": "Adam Bandt",
    "One Nation": "Pauline Hanson",
    "Australia's Voice": "Fatima Payman",
    "Centre Alliance": "Rebekha Sharkie",
    "David Pocock": "David Pocock",
    "Jacqui Lambie Network": "Jacqui Lambie",
    "Katter's Australian Party": "Robbie Katter",
    "People First Party": "Gerard Rennick",
    "United Australia Party": "Ralph Babet",
    "Animal Justice Party": "Angela Pollard",
    "Australian Christians": "Maryka Groenewald",
    "Australian Citizens Party": "Craig Isherwood",
    "Australian Democrats": "Lyn Allison",
    "Better Together Party": "Lucy Bradlow",
    "Fusion Party": "Drew Wolfendale",
    "Family First Party": "Lyle Shelton",
    "The Great Australian Party": "Rod Culleton",
    "Indigenous-Aboriginal Party": "'Uncle' Owen Whyman",
    "Kim for Canberra": "Kim Rubenstein",
    "Legalise Cannabis": "Michael Balderstone",
    "Libertarian Party": "Anthony Bull",
    "Shooters, Fishers, and Farmers Party": "Robert Brown",
    "Socialist Alliance": "Jacob Andrewartha",
    "Sustainable Australia Party": "Celeste Ackerly",
    "Trumpet of Patriots": "Suellen Wrightson",
    "Victorian Socialists": "Collective leadership",
    "Katter's Australian Party": "Bob Katter",
    "Independents": "Independents"
}
abbreviations_parties = {
    "Greens": "Australian Greens",
    "Labor": "Australian Labor Party",
    "Liberal": "Coalition",
    "National": "Coalition"
}
abbreviations_leaders = {
    "Dutton": "Peter Dutton",
    "Albanese": "Anthony Albanese",
    "Bandt": "Adam Bandt"
}

In [49]:
def add_party_and_leader_entities(nlp):
    """Creates a rule-based system for important entities"""
    # Creating a new named entity ruler before the NER pipe
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    
    # Prepare a list of patterns for full names and abbreviations
    patterns = []
    
    # Add patterns for full party names and their leaders
    for party, leader in party_leader_pairs.items():
        patterns.append({"label": "ORG", "pattern": party})  # Party as ORG
        patterns.append({"label": "PERSON", "pattern": leader})  # Leader as PERSON
    
    # Add patterns for abbreviations
    for short_party_name, full_party_name in abbreviations_parties.items():
        patterns.append({"label": "ORG", "pattern": short_party_name})
    
    for short_name, full_name in abbreviations_leaders.items():
        patterns.append({"label": "PERSON", "pattern": short_name}) 
    
    # Add the patterns to the entity ruler
    ruler.add_patterns(patterns)


In [56]:
def ner_with_spacy(text):
    nlp = spacy.load("en_core_web_sm")
    add_party_and_leader_entities(nlp)
    people = []
    parties = []
    doc = nlp(text)
    
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            people.append(ent.text)
        elif ent.label_ == "ORG":
            if ent.text in party_leader_pairs.keys() or ent.text in abbreviations_parties.keys():
                parties.append(ent.text)
    return people, parties


In [57]:
# test
text = "Albanese met with Dutton and Adam Bandt to discuss policies. The Labor Party and Greens were in agreement."
people, parties = ner_with_spacy(text)

print("People:", people)
print("Parties:", parties)

People: ['Albanese', 'Dutton', 'Adam Bandt']
Parties: ['Labor', 'Greens']


In [58]:
corpus_df[['headline_people', 'headline_parties']] = corpus_df['headline'].apply(lambda x: pd.Series(ner_with_spacy(x)))

In [59]:
corpus_df[['content_people', 'content_parties']] = corpus_df['content'].apply(lambda x: pd.Series(ner_with_spacy(x)))

In [61]:
corpus_df['headline_people'] = corpus_df['headline_people'].apply(lambda x: list(set(x)))
corpus_df['headline_parties'] = corpus_df['headline_parties'].apply(lambda x: list(set(x)))
corpus_df['content_people'] = corpus_df['content_people'].apply(lambda x: list(set(x)))
corpus_df['content_parties'] = corpus_df['content_parties'].apply(lambda x: list(set(x)))

In [62]:
corpus_df

,headline,title,content,date,url,source,content_length,headline_people,headline_parties,content_people,content_parties
0,New polling reveals Jacinta Allan’s support pl...,Polls reveals sharp drop in support for Jacint...,New polling has revealed support for Jacinta A...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews,406,[],[],"[Daniel Andrews, Andrews, Ms Allan, Allan, Jim...","[Coalition, Labor, Liberal]"
1,Albanese’s major election push for group of Au...,Federal Election 2025: Labor to make submissio...,Labor will push the Fair Work Commission to gi...,2025-04-02,https://www.skynews.com.au/australia-news/poli...,skynews,424,[Albanese],[],"[Albanese, Anthony Albanese, Dutton, Peter Dut...","[Labor, Coalition, Australian Labor Party, Lib..."
2,Albanese hits back at Trump’s list of grievanc...,Anthony Albanese pressed on relationship with ...,Prime Minister Anthony Albanese has insisted h...,2025-04-01,https://www.skynews.com.au/australia-news/poli...,skynews,457,[Albanese],[],"[Albanese, Trump, Anthony Albanese, Donald Trump]",[Labor]
3,Bipartisan support to invest $150m for health ...,Coalition to match Labor's $150 million pledge...,A $150 million investment into a new healthcar...,2025-04-01,https://www.abc.net.au/news/2025-03-31/federal...,abc,749,[],[],"[Colin Stirling, Anthony Albanese, Mark Butler...","[Labor, Coalition, Liberal]"
4,Coalition promises to relax home lending rules...,Coalition promises to relax home lending rules...,The Coalition is promising to relax rules arou...,2025-04-01,https://www.abc.net.au/news/2025-04-01/electio...,abc,659,[],[Coalition],"[Anthony Albanese, Peter Dutton, Michael Sukka...","[Coalition, National, Greens]"
...,...,...,...,...,...,...,...,...,...,...,...
497,Albanese unveils $7.2b highway upgrade as elec...,Federal Labor pledges to fund bulk of Bruce Hi...,The federal government has announced it will f...,2025-01-06,https://www.abc.net.au/news/2025-01-06/federal...,abc,976,[Albanese],[],"[Bruce Highway, Cairns, Anthony Albanese, Bena...","[Labor, Coalition]"
498,Labor plunges $7.2bn to upgrade one of Austral...,Albanese government gives $7.2 billion funding...,Prime Minister Anthony Albanese has used his f...,2025-01-06,https://www.skynews.com.au/australia-news/poli...,skynews,553,[],[Labor],"[Bruce Highway, Cairns, Anthony Albanese, Cath...",[Liberal]
499,Anthony Albanese switches to election footing ...,Anthony Albanese switches to election footing ...,PM frames federal poll as choice between ‘buil...,2025-01-06,https://www.theguardian.com/australia-news/202...,theguardian,561,[Anthony Albanese],[],"[Cairns, Anthony Albanese, Albanese, Warren En...","[Labor, Coalition, Liberal]"
500,‘A huge hit for any family’: New emissions sta...,‘A huge hit for any family’: Coalition warns n...,The federal Coalition is warning Australian co...,2025-01-02,https://www.skynews.com.au/australia-news/poli...,skynews,402,[jack],[],"[Albanese, Jono Duniam, Duniam, David Littlepr...","[Coalition, Labor, Liberal]"


In [70]:
corpus_df.to_csv('corpus_NER.csv', index=False)

In [ ]:
corpus_df = pd.read.csv